Cryptocompare  
https://developers.cryptocompare.com/documentation/data-api/spot_v1_historical_hours

In [18]:
import os
import time
import requests
import pandas as pd
from datetime import datetime, timedelta

API_KEY = "83b1fe1bf4762e0bfb056afa64f89b227988cd131d26ee0ff465b9c3a7070e19" # cryptocompare api key

In [19]:
url_markets = "https://data-api.cryptocompare.com/spot/v1/markets"

headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}
response = requests.get(url_markets, headers=headers)

if response.status_code == 200:
    data = response.json()["Data"]

    # **提取所有 market（交易所）**
    markets = [{"market": market, **details} for market, details in data.items()]
    markets_df = pd.DataFrame(markets)
    print(f"✅ 获取 {len(markets_df)} 个交易所")

else:
    print(f"❌ API 请求失败: {response.status_code}, {response.text}")

markets_df.head()

✅ 获取 176 个交易所


,market,TYPE,EXCHANGE_STATUS,MAPPED_INSTRUMENTS_TOTAL,UNMAPPED_INSTRUMENTS_TOTAL,INSTRUMENT_STATUS,TOTAL_TRADES_SPOT,HAS_ORDERBOOK_L2_MINUTE_SNAPSHOTS_ENABLED
0,aax,602,ACTIVE,250,185,"{'ACTIVE': 88, 'IGNORED': 0, 'RETIRED': 347, '...",648891006,False
1,abcc,602,ACTIVE,304,28,"{'ACTIVE': 29, 'IGNORED': 0, 'RETIRED': 303, '...",8299545,False
2,acx,602,ACTIVE,0,32,"{'ACTIVE': 32, 'IGNORED': 0, 'RETIRED': 0, 'EX...",6203612,False
3,aidosmarket,602,ACTIVE,1,2,"{'ACTIVE': 2, 'IGNORED': 0, 'RETIRED': 1, 'EXP...",155608,False
4,alphaex,602,ACTIVE,10,2,"{'ACTIVE': 2, 'IGNORED': 0, 'RETIRED': 10, 'EX...",1718824,False


In [20]:
url_instruments = "https://data-api.cryptocompare.com/spot/v1/markets/instruments"
headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}

all_instruments = []

for market in markets_df["market"]:
    params = {"market": market}
    response = requests.get(url_instruments, headers=headers, params=params)

    json_response = response.json()
    market_data = json_response["Data"].get(market, {})
    instruments_data = market_data.get("instruments", {})

    # 遍历market的所有交易对
    for instrument_name, instrument_details in instruments_data.items():
        all_instruments.append({
            "market": market,
            "instrument": instrument_name,
            "base_asset": instrument_details.get("INSTRUMENT", "N/A"),
            "status": instrument_details.get("INSTRUMENT_STATUS", "UNKNOWN")
        })

    print(f"✅ {market} 交易所的 {len(instruments_data)} 个交易对已获取")

    # 避免过快请求
    time.sleep(0.5)

instruments_df = pd.DataFrame(all_instruments)
instruments_df.head()

save_dir = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result"
csv_filepath = os.path.join(save_dir, "crypto_instruments.csv")
instruments_df.to_csv(csv_filepath, index=False, encoding="utf-8")


✅ aax 交易所的 0 个交易对已获取
✅ abcc 交易所的 1 个交易对已获取
✅ acx 交易所的 0 个交易对已获取
✅ aidosmarket 交易所的 0 个交易对已获取
✅ alphaex 交易所的 0 个交易对已获取
✅ archax 交易所的 0 个交易对已获取
✅ ascendex 交易所的 532 个交易对已获取
✅ ataix 交易所的 0 个交易对已获取
✅ backpack 交易所的 55 个交易对已获取
✅ bequant 交易所的 190 个交易对已获取
✅ bgogo 交易所的 0 个交易对已获取
✅ bibox365 交易所的 263 个交易对已获取
✅ bigone 交易所的 145 个交易对已获取
✅ bilaxy 交易所的 343 个交易对已获取
✅ binance 交易所的 1339 个交易对已获取
✅ binanceaggregate 交易所的 1339 个交易对已获取
✅ binanceusa 交易所的 183 个交易对已获取
✅ bingx 交易所的 882 个交易对已获取
✅ bisq 交易所的 0 个交易对已获取
✅ bit 交易所的 68 个交易对已获取
✅ bit2c 交易所的 4 个交易对已获取
✅ bit2me 交易所的 329 个交易对已获取
✅ bitbank 交易所的 42 个交易对已获取
✅ bitbay 交易所的 0 个交易对已获取
✅ bitbns 交易所的 158 个交易对已获取
✅ bitbuy 交易所的 0 个交易对已获取
✅ bitci 交易所的 143 个交易对已获取
✅ bitexbook 交易所的 33 个交易对已获取
✅ bitfex 交易所的 0 个交易对已获取
✅ bitfinex 交易所的 285 个交易对已获取
✅ bitflyer 交易所的 7 个交易对已获取
✅ bitflyereu 交易所的 4 个交易对已获取
✅ bitflyerfx 交易所的 1 个交易对已获取
✅ bitflyerus 交易所的 4 个交易对已获取
✅ bitforex 交易所的 0 个交易对已获取
✅ bitget 交易所的 787 个交易对已获取
✅ bithumbglobal 交易所的 0 个交易对已获取
✅ bithumbkorea 交易所的 342 个交易对已获取
✅ bitin

In [24]:
url_spot_volume = 'https://data-api.cryptocompare.com/spot/v1/historical/hours'
eaders = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}

trade_data = []

# 遍历exchanges&instruments
for _, row in instruments_df.iterrows():
    market = row["market"]
    instrument = row["instrument"]

    params = {
        "market": market,
        "instrument": instrument,
        "limit": 1,  # 获取最近 1 小时数据
        "aggregate": 1,
        "fill": "true",
        "apply_mapping": "true",
        "response_format": "JSON",
        "groups": "TRADE"
    }

    response = requests.get(url_spot_volume, headers=headers, params=params)
    response.raise_for_status()
    json_response = response.json()

    # **检查数据是否有效**
    if "Data" in json_response and isinstance(json_response["Data"], list) and len(json_response["Data"]) > 0:
        entry = json_response["Data"][0]  # 获取最近的交易量数据点
        trade_data.append({
            "market": market,
            "instrument": instrument,
            "time": pd.to_datetime(entry["TIMESTAMP"], unit="s"),
            "total_trades": entry["TOTAL_TRADES"],
            "total_trades_buy": entry.get("TOTAL_TRADES_BUY", 0),
            "total_trades_sell": entry.get("TOTAL_TRADES_SELL", 0)
        })
        print(f"✅ Data successfully fetched: {market} - {instrument}")
    else:
        print(f"❌ No data for {market} - {instrument}")

    time.sleep(0.5)

trade_volume_df = pd.DataFrame(trade_data)
pivot_df = trade_volume_df.pivot(index="instrument", columns="market", values="total_trades")

✅ Data successfully fetched: abcc - BTC-USDT
✅ Data successfully fetched: ascendex - GOAT-USDT
✅ Data successfully fetched: ascendex - MAHA-USDT
✅ Data successfully fetched: ascendex - XTZ-BTC
✅ Data successfully fetched: ascendex - MOTH-USDT
✅ Data successfully fetched: ascendex - BNB-USD
✅ Data successfully fetched: ascendex - HTR-USDT
✅ Data successfully fetched: ascendex - RSIC-USDT
✅ Data successfully fetched: ascendex - NIM-USDT
✅ Data successfully fetched: ascendex - ZEC-USDT
✅ Data successfully fetched: ascendex - YCT-USDT
✅ Data successfully fetched: ascendex - MIM-USDT
✅ Data successfully fetched: ascendex - SUSHI-USDT
✅ Data successfully fetched: ascendex - ZEE-USDT
✅ Data successfully fetched: ascendex - ROSS-USDT
✅ Data successfully fetched: ascendex - PEPE-USDT
✅ Data successfully fetched: ascendex - DOT-USD
✅ Data successfully fetched: ascendex - POOLX-USDT
✅ Data successfully fetched: ascendex - BTC-USD
✅ Data successfully fetched: ascendex - SWP-USDT
✅ Data successfull

KeyboardInterrupt: 

CoinApi  
https://docs.coinapi.io/  
time_start 和 time_end 不能超过 1 天，所以你只能请求过去 24 小时以内的 hourly 数据。
API_KEY = "790215ee-49aa-4240-bf83-421ace0dfb76"  
url = "https://rest.coinapi.io/v1/exchanges"